In [7]:
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
from langchain_core import tools
from langchain.tools import tool
from IPython.display import Markdown


load_dotenv("./.env")

llm = ChatOpenAI(model="gpt-4o")



In [ ]:
@tool 
def write_email(to: str, subject:str, content: str) -> str:
    """Write and send email"""
    return f"Email sent to {to} with subject {subject} and content : {content}"

model_with_tools = llm.bind_tools( [write_email] , parallel_tool_calls=False)
output = model_with_tools.invoke("Envia un email a manuonda@gmail.com diciendo que es un crack")

print(output.tool_calls[0])
print(f"Name tool : {output.tool_calls[0]['name']}")

type(output)
args = output.tool_calls[0]['args']
print(f"Args : {args}")

#call the tool
result = write_email.invoke(args)
Markdown(result)


{'name': 'write_email', 'args': {'to': 'manuonda@gmail.com', 'subject': '¡Eres un crack!', 'content': '¡Hola! Solo quería tomarme un momento para decirte que eres un crack. Sigue haciendo ese gran trabajo y esperamos ver todo lo increíble que lograrás. ¡Un fuerte abrazo!'}, 'id': 'call_KtWaQdcuJPAdMOEKojg69E4U', 'type': 'tool_call'}
Name tool : write_email
Args : {'to': 'manuonda@gmail.com', 'subject': '¡Eres un crack!', 'content': '¡Hola! Solo quería tomarme un momento para decirte que eres un crack. Sigue haciendo ese gran trabajo y esperamos ver todo lo increíble que lograrás. ¡Un fuerte abrazo!'}


Email sent to manuonda@gmail.com with subject ¡Eres un crack! and content : ¡Hola! Solo quería tomarme un momento para decirte que eres un crack. Sigue haciendo ese gran trabajo y esperamos ver todo lo increíble que lograrás. ¡Un fuerte abrazo!

In [ ]:
from typing import TypedDict
from langgraph.graph  import START, END, StateGraph

class StateSchema(TypedDict):
    request: str
    email: str
    
workflow = StateGraph(StateSchema)

def write_email(state: StateSchema)-> StateSchema:
    """ Imperative code that processes the request """
    output = model_with_tools.invoke(state['request'])
    args = output.tool_calls['0']['args']
    email = write_email.invoke(args)
    return {"email": email}


workflow.add_node("write_email")
